In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [8]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

# Test tempview
spark.sql("SELECT * FROM home_sales LIMIT 10").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [16]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT Round(avg(price),2) FROM home_sales WHERE bedrooms == 4").show()

+--------------------+
|round(avg(price), 2)|
+--------------------+
|           299661.01|
+--------------------+



In [23]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT date_built, Round(avg(price),2) as average_price
FROM home_sales 
WHERE bedrooms == 3 AND bathrooms == 3
GROUP BY date_built
ORDER BY date_built ASC""").show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [24]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""SELECT date_built, Round(avg(price),2) as average_price
FROM home_sales 
WHERE bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built ASC""").show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [26]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
spark.sql("""SELECT date_built, 
Round(avg(price),2) as average_price,
Round(avg(view),2) as average_view
FROM home_sales 
WHERE price >= 350000
GROUP BY date_built
ORDER BY date_built ASC""").show()

start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+-------------+------------+
|date_built|average_price|average_view|
+----------+-------------+------------+
|      2010|    463301.08|       30.64|
|      2011|    472861.91|       32.28|
|      2012|    475524.98|       33.48|
|      2013|    476837.88|       32.61|
|      2014|    475674.82|       32.28|
|      2015|    471517.37|       31.89|
|      2016|    478550.09|       32.91|
|      2017|    475884.72|       32.03|
+----------+-------------+------------+

--- 0.00020432472229003906 seconds ---


In [27]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [28]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [29]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
spark.sql("""SELECT date_built, 
Round(avg(price),2) as average_price,
Round(avg(view),2) as average_view
FROM home_sales 
WHERE price >= 350000
GROUP BY date_built
ORDER BY date_built ASC""").show()


start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))


+----------+-------------+------------+
|date_built|average_price|average_view|
+----------+-------------+------------+
|      2010|    463301.08|       30.64|
|      2011|    472861.91|       32.28|
|      2012|    475524.98|       33.48|
|      2013|    476837.88|       32.61|
|      2014|    475674.82|       32.28|
|      2015|    471517.37|       31.89|
|      2016|    478550.09|       32.91|
|      2017|    475884.72|       32.03|
+----------+-------------+------------+

--- 0.00012254714965820312 seconds ---


In [30]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.parquet('parquet_home_sales', mode='overwrite')
p_df=spark.read.parquet('parquet_home_sales')
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [31]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('home_sales_partitioned')

In [32]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('home_sales_p')

In [33]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.
spark.sql("""SELECT date_built, 
Round(avg(price),2) as average_price,
Round(avg(view),2) as average_view
FROM home_sales_p 
WHERE price >= 350000
GROUP BY date_built
ORDER BY date_built ASC""").show()


start_time = time.time()


print("--- %s seconds ---" % (time.time() - start_time))

+----------+-------------+------------+
|date_built|average_price|average_view|
+----------+-------------+------------+
|      2010|    463301.08|       30.64|
|      2011|    472861.91|       32.28|
|      2012|    475524.98|       33.48|
|      2013|    476837.88|       32.61|
|      2014|    475674.82|       32.28|
|      2015|    471517.37|       31.89|
|      2016|    478550.09|       32.91|
|      2017|    475884.72|       32.03|
+----------+-------------+------------+

--- 0.0002529621124267578 seconds ---


In [34]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [35]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False